In [1]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql import SQLContext, Row
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from pyspark.ml import Pipeline
#from py4j.java_gateway import java_import
import sparknlp
#sc = sparknlp.start()
#sc.stop()

In [2]:
sc = SparkContext()
sqlContext = SQLContext(sc)


/home/yogender/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning
/home/yogender/.local/lib/python3.6/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [3]:
rdd_l = sc.wholeTextFiles("./*.html")

In [4]:
rdd1 = rdd_l.map(lambda x: x[1]).map(lambda x: x.split('\n\t')).map(lambda x: x[6]) \
.map(lambda x: x.split('prodeji')).map(lambda x: x[1]).map(lambda x: x.split(';')) \
.map(lambda x: x[0]).map(lambda x: x.strip(' '))

rows_split_addr = rdd1.map(lambda p: Row(p))

print(rows_split_addr.collect())

schemaString = "address"
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
addressSchema = sqlContext.createDataFrame(rows_split_addr, schema)
#print(addressSchema.collect())
addressSchema.repartition(1).write.csv(path='./data.csv', mode="append", header="true")
addressSchema.collect()

[<Row('Masná, Praha 1 - Staré Město')>, <Row('Velvarská, Horoměřice')>, <Row('Libušina, Karlovy Vary')>, <Row('Pod Lipami, Praha 3 - Žižkov')>, <Row('Drnovská, Praha 6 - Ruzyně')>, <Row('V Průhoně, Mukařov - Žernovka')>, <Row('Mladoboleslavská, Mělník')>, <Row('Grafická, Praha 5 - Smíchov')>, <Row('Sametová, Liberec - Liberec VI-Rochlice')>, <Row('Jakubská, Praha 1 - Staré Město')>]


[Row(address='Masná, Praha 1 - Staré Město'),
 Row(address='Velvarská, Horoměřice'),
 Row(address='Libušina, Karlovy Vary'),
 Row(address='Pod Lipami, Praha 3 - Žižkov'),
 Row(address='Drnovská, Praha 6 - Ruzyně'),
 Row(address='V Průhoně, Mukařov - Žernovka'),
 Row(address='Mladoboleslavská, Mělník'),
 Row(address='Grafická, Praha 5 - Smíchov'),
 Row(address='Sametová, Liberec - Liberec VI-Rochlice'),
 Row(address='Jakubská, Praha 1 - Staré Město')]

In [5]:
address_pd_df = addressSchema.toPandas()

In [6]:
address_pd_df

,address
0,"Masná, Praha 1 - Staré Město"
1,"Velvarská, Horoměřice"
2,"Libušina, Karlovy Vary"
3,"Pod Lipami, Praha 3 - Žižkov"
4,"Drnovská, Praha 6 - Ruzyně"
5,"V Průhoně, Mukařov - Žernovka"
6,"Mladoboleslavská, Mělník"
7,"Grafická, Praha 5 - Smíchov"
8,"Sametová, Liberec - Liberec VI-Rochlice"
9,"Jakubská, Praha 1 - Staré Město"


In [7]:
sc.stop()

In [8]:
spark= sparknlp.start()

/home/yogender/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [9]:
sqlContext = SQLContext(spark)

/home/yogender/.local/lib/python3.6/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [10]:
rdd_l = spark.sparkContext.wholeTextFiles("./*.html")

In [11]:
df = rdd_l.toDF(schema=["filename", "text"]).select("text")

In [12]:
rules = '''

\d+&\w+;\d+&\w+;Kč* 

'''
#\S*\d+\S* Kč* 
with open('regex_rules.txt', 'w') as f:
    
    f.write(rules)

In [15]:
from py4j.java_gateway import java_import

In [16]:
RegexMatcher().extractParamMap()

TypeError: 'JavaPackage' object is not callable

In [18]:
from py4j.java_gateway import java_import
java_import(spark._sc._jvm, "org.apache.spark.sql.api.python.*")

In [19]:




documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("assembled")

regex_matcher = RegexMatcher()\
    .setInputCols('assembled')\
    .setStrategy("MATCH_ALL")\
    .setOutputCol("regex_matches")\
    .setExternalRules(path='./regex_rules.txt', delimiter=',')

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    regex_matcher
 ])

pipelineModel = nlpPipeline.fit(df)
#df = pipelineModel.transform(df)
#empty_df = spark.createDataFrame([['']]).toDF("text")

#pipelineModel = nlpPipeline.fit(empty_df)

match_df = pipelineModel.transform(df)

match_df.select('regex_matches.result').show(truncate=50)

TypeError: 'JavaPackage' object is not callable

In [47]:
from pyspark.sql.functions import regexp_replace, col
import re

In [101]:
match_df_flat = match_df.select("regex_matches.result").rdd.flatMap(lambda x: x[0])

In [167]:
match_df_flat.collect()

['500&nbsp;000&nbsp;Kč',
 '200&nbsp;000&nbsp;Kč',
 '854&nbsp;900&nbsp;Kč',
 '565&nbsp;000&nbsp;Kč',
 '475&nbsp;000&nbsp;Kč',
 '500&nbsp;000&nbsp;Kč',
 '530&nbsp;000&nbsp;Kč',
 '511&nbsp;000&nbsp;Kč',
 '850&nbsp;000&nbsp;Kč',
 '490&nbsp;000&nbsp;Kč']

In [185]:
match_df_flat1 = match_df_flat.map(lambda s: s.replace('900&nbsp', 'x')).map(lambda s: s.replace(';', ''))\
.map(lambda s: s.replace('&nbsp000&nbs', '0000')) \
.map(lambda s: s.replace('&nbspx', '9000')) \
.map(lambda s: s.replace('p', '')) \
.map(lambda s: s.replace('Kč', ' Kč')) \
.map(lambda s: s.replace('Kč', ' Kč')) \

In [186]:
#match_df_flat1 = match_df_flat.map(lambda s: s.replace('&nbsp', ' ')).map(lambda s: s.replace(';', '')) \
#.map(lambda s: s.replace('000', '0000')) \

In [187]:
match_df_flat1.collect()

['5000000 Kč',
 '2000000 Kč',
 '8549000 Kč',
 '5650000 Kč',
 '4750000 Kč',
 '5000000 Kč',
 '5300000 Kč',
 '5110000 Kč',
 '8500000 Kč',
 '4900000 Kč']

In [188]:
#match_df.withColumn("result", re.sub(r"&", "0", match_df)).show()

In [189]:
#m = match_df.select('regex_matches.result').replace('&','0')


In [190]:
match_df = match_df.rdd.flatMap(lambda xs: [x[0] for x in xs])

In [191]:
#match_df.toDF(schema=["filename", "text"]).select("text")

In [192]:
#match_df.select('regex_matches.result').show()

In [193]:
#re.sub("$", '', match_df_flat)

In [194]:
#m.select('regex_matches.result').show()

In [196]:
#match_df_flat.withColumn("result", regexp_replace("result","&","0")).show()

In [197]:
from pyspark.sql import Row

row = Row("cost") 
match_df_flat1_df= match_df_flat1.map(row).toDF()


In [148]:
#match_df_flat1 = match_df_flat1.toDF()

In [198]:
match_df_pd_cost = match_df_flat1_df.toPandas()


In [199]:
match_df_pd_cost

,cost
0,5000000 Kč
1,2000000 Kč
2,8549000 Kč
3,5650000 Kč
4,4750000 Kč
5,5000000 Kč
6,5300000 Kč
7,5110000 Kč
8,8500000 Kč
9,4900000 Kč


In [200]:
import pandas as pd
df_concat_cost_address = pd.concat([match_df_pd_cost,address_pd_df], axis=1)

In [201]:
df_concat_cost_address

,cost,address
0,5000000 Kč,"Masná, Praha 1 - Staré Město"
1,2000000 Kč,"Velvarská, Horoměřice"
2,8549000 Kč,"Libušina, Karlovy Vary"
3,5650000 Kč,"Pod Lipami, Praha 3 - Žižkov"
4,4750000 Kč,"Drnovská, Praha 6 - Ruzyně"
5,5000000 Kč,"V Průhoně, Mukařov - Žernovka"
6,5300000 Kč,"Mladoboleslavská, Mělník"
7,5110000 Kč,"Grafická, Praha 5 - Smíchov"
8,8500000 Kč,"Sametová, Liberec - Liberec VI-Rochlice"
9,4900000 Kč,"Jakubská, Praha 1 - Staré Město"


In [202]:
df_concat_cost_address.to_csv('cost_address', sep=',', index=False)

In [1]:
df_cost_address = pd.read_csv('./cost_address', sep=',')

NameError: name 'pd' is not defined

In [205]:
df_

,cost,address
0,5000000 Kč,"Masná, Praha 1 - Staré Město"
1,2000000 Kč,"Velvarská, Horoměřice"
2,8549000 Kč,"Libušina, Karlovy Vary"
3,5650000 Kč,"Pod Lipami, Praha 3 - Žižkov"
4,4750000 Kč,"Drnovská, Praha 6 - Ruzyně"
5,5000000 Kč,"V Průhoně, Mukařov - Žernovka"
6,5300000 Kč,"Mladoboleslavská, Mělník"
7,5110000 Kč,"Grafická, Praha 5 - Smíchov"
8,8500000 Kč,"Sametová, Liberec - Liberec VI-Rochlice"
9,4900000 Kč,"Jakubská, Praha 1 - Staré Město"
